In [574]:
import pandas as pd
import numpy as np

In [601]:
dataset = pd.read_csv('Data/volume_of_disc.csv')
dataset = dataset.drop('Unnamed: 0', axis =1).fillna(0)
volume = pd.read_csv('Data/data_grid_dataset.csv')
volume = volume.fillna(0)
volume['created_at'] = pd.to_datetime(volume['created_at']).dt.date

In [605]:
dataset[['volume', 'disc1', 'disc2', 'disc3', 'disc4', 'disc5']].tail(5)

,volume,disc1,disc2,disc3,disc4,disc5
1883,27,13.0,12.0,2.0,0.0,0.0
1884,17,6.0,10.0,1.0,0.0,0.0
1885,20,5.0,14.0,1.0,0.0,0.0
1886,62,11.0,45.0,5.0,1.0,0.0
1887,45,11.0,31.0,3.0,0.0,0.0


In [578]:
disc_list = ['disc1','disc2','disc3', 'disc4', 'disc5']

for i in disc_list:
    dataset['norm' + str(i)] = ((dataset[i] / dataset.volume)*100).round(-1)

In [599]:
dataset

,created_at,volume,revenue,full_revenue,disc1,disc2,disc3,disc4,disc5,normdisc1,normdisc2,normdisc3,normdisc4,normdisc5
0,2013-08-28,1,1695,1695,1.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
1,2013-12-15,1,3163,4001,0.0,0.0,1.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
2,2013-12-23,2,8000,8000,2.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
3,2014-02-07,1,4000,4000,1.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
4,2014-03-05,1,2795,2795,1.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883,2020-09-11,27,84095,91200,13.0,12.0,2.0,0.0,0.0,50.0,40.0,10.0,0.0,0.0
1884,2020-09-12,17,44905,51100,6.0,10.0,1.0,0.0,0.0,40.0,60.0,10.0,0.0,0.0
1885,2020-09-13,20,57465,64000,5.0,14.0,1.0,0.0,0.0,20.0,70.0,0.0,0.0,0.0
1886,2020-09-14,62,173740,204300,11.0,45.0,5.0,1.0,0.0,20.0,70.0,10.0,0.0,0.0


In [579]:
grid_dataset = (dataset[['normdisc1','normdisc2','normdisc3','normdisc4', 'normdisc5']]
                .drop_duplicates().reset_index().drop('index', axis = 1))

In [580]:
grid_dataset['grid_id'] = grid_dataset.index

In [597]:
grid_dataset.head(5)

,normdisc1,normdisc2,normdisc3,normdisc4,normdisc5,grid_id
0,100.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,100.0,0.0,0.0,1
2,0.0,100.0,0.0,0.0,0.0,2
3,0.0,0.0,0.0,100.0,0.0,3
4,0.0,0.0,0.0,0.0,0.0,4


In [581]:
feature_list = ['created_at','campaign_type_1','opened_1','camp_volume_1','users_received_8','campaign_type_8'
                ,'disc1_pre_3days','disc1_pre_6day','disc1_pre_14dat',
                'disc2_pre_4days','disc2_pre_14day','disc3_pre_1day', 'disc4_pre_2day',
                 'disc5_pre_13day','products_pre_7day','products_pre_14day','new_users_pre_3days',
                 'volume_pre_3days','volume_pre_7day','volume_pre_14day','full_revenue_pre_7day',
                    'year','month', 'volume']

volume = volume[feature_list]

In [582]:
volume['volume_pre_3days'] = volume.sort_values('created_at').volume.shift(1).rolling(3).sum()
volume['volume_pre_4days'] = volume.sort_values('created_at').volume.shift(1).rolling(4).sum()
volume['volume_pre_6day'] = volume.sort_values('created_at').volume.shift(6)
volume['volume_pre_14day'] = volume.sort_values('created_at').volume.shift(14)
volume['volume_pre_7day'] = volume.sort_values('created_at').volume.shift(7)
volume['volume_pre_1day'] = volume.sort_values('created_at').volume.shift(1)
volume['volume_pre_2day'] = volume.sort_values('created_at').volume.shift(2)
volume['volume_pre_13day'] = volume.sort_values('created_at').volume.shift(13)

In [583]:
price_opt_dataset = volume.merge(grid_dataset, how = 'cross')

In [584]:
price_opt_dataset['disc1_pre_3days'] =(( price_opt_dataset['disc1_pre_3days']/price_opt_dataset['volume_pre_3days']
                                        * price_opt_dataset.normdisc1 / 100).round(0))

price_opt_dataset['disc1_pre_14dat'] =(( price_opt_dataset['disc1_pre_14dat']/price_opt_dataset['disc1_pre_14dat']
                                        * price_opt_dataset.normdisc1 / 100).round(0))


price_opt_dataset['disc1_pre_6day'] =(( price_opt_dataset['disc1_pre_6day']/price_opt_dataset['disc1_pre_6day']
                                        * price_opt_dataset.normdisc1 / 100).round(0))


price_opt_dataset['disc2_pre_4days'] =(( price_opt_dataset['disc2_pre_4days']/price_opt_dataset['disc2_pre_4days']
                                        * price_opt_dataset.normdisc2 / 100).round(0))


price_opt_dataset['disc2_pre_14day'] =(( price_opt_dataset['disc2_pre_14day']/price_opt_dataset['disc2_pre_14day']
                                        * price_opt_dataset.normdisc2 / 100).round(0))


price_opt_dataset['disc3_pre_1day'] =(( price_opt_dataset['disc3_pre_1day']/price_opt_dataset['disc3_pre_1day']
                                        * price_opt_dataset.normdisc3 / 100).round(0))


price_opt_dataset['disc4_pre_2day'] =(( price_opt_dataset['disc4_pre_2day']/price_opt_dataset['disc4_pre_2day']
                                        * price_opt_dataset.normdisc4 / 100).round(0))


price_opt_dataset['disc5_pre_13day'] =(( price_opt_dataset['disc5_pre_13day']/price_opt_dataset['disc5_pre_13day']
                                        * price_opt_dataset.normdisc5 / 100).round(0))

In [585]:
opt_list = ['campaign_type_1','opened_1','camp_volume_1','users_received_8','campaign_type_8'
                ,'disc1_pre_3days','disc1_pre_6day','disc1_pre_14dat',
                'disc2_pre_4days','disc2_pre_14day','disc3_pre_1day',
                 'disc5_pre_13day','products_pre_7day','products_pre_14day','new_users_pre_3days',
                 'volume_pre_3days','volume_pre_7day','volume_pre_14day','full_revenue_pre_7day',
                    'year','month', 'grid_id']

price_opt_dataset= price_opt_dataset[opt_list].replace([np.inf, -np.inf], 0).fillna(0)

In [619]:
price_opt_dataset[price_opt_dataset.year == 2020].sample(5)

,campaign_type_1,opened_1,camp_volume_1,users_received_8,campaign_type_8,disc1_pre_3days,disc1_pre_6day,disc1_pre_14dat,disc2_pre_4days,disc2_pre_14day,...,products_pre_7day,products_pre_14day,new_users_pre_3days,volume_pre_3days,volume_pre_7day,volume_pre_14day,full_revenue_pre_7day,year,month,grid_id
267814,26.0,88.0,209.0,145.0,23.0,0.0,0.0,0.0,0.0,0.0,...,18.0,13.0,10.0,61.0,23.0,19.0,74900.0,2020,2,115
286918,17.0,53.0,91.0,241.0,28.0,0.0,0.0,0.0,0.0,0.0,...,19.0,23.0,24.0,107.0,69.0,85.0,231800.0,2020,5,33
276612,25.0,86.0,149.0,221.0,33.0,0.0,0.0,0.0,0.0,0.0,...,21.0,31.0,27.0,111.0,37.0,139.0,121400.0,2020,3,44
265428,26.0,249.0,1425.0,2490.0,30.0,0.0,0.0,0.0,0.0,0.0,...,12.0,14.0,9.0,42.0,23.0,23.0,71900.0,2020,1,82
291263,28.0,654.0,2563.0,2483.0,25.0,0.0,0.0,0.0,0.0,0.0,...,15.0,16.0,17.0,134.0,36.0,37.0,116900.0,2020,6,34


In [587]:
opt_price = {}
for i in range(price_opt_dataset.grid_id.max()):
    opt_price[str(i)] = price_opt_dataset[price_opt_dataset.grid_id == i] 
    opt_price[str(i)].to_csv('Data/opt_price' + str(i) + '.csv')

In [595]:
grid_dataset.to_csv('Data/grid_dataset.csv')